## Import Library, Models and Data

#### Load Libraries

In [1]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -qq install --upgrade --force-reinstall llama-cpp-python==0.1.78 --no-cache-dir

In [2]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.18-cp310-cp310-manylinux_2_35_x86_64.whl size=1954339 sha256=5c5ad1d505918fcc79a925bbdd06e0336f2eb44a139c68bc75de13f7fd00a119
  Stored in directory: /root/.cache/pip/wheels/75/65/c3/54e9fca551b4734cc17f41fafb596a7807312fb470c806e1ab
Successfully built llama-cpp-python


In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00


In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 2

In [5]:
!pip install langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 4.6 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import textwrap

In [7]:
# for document processing
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# for generating embedding and storing in vectorDB
from langchain.vectorstores import Chroma
# from langchain.embeddings import GPT4AllEmbeddings

# for setting up the model
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# chaining the buffer, prompt template
from langchain.chains import LLMChain

In [8]:
# for setting up LLaMa model
from os.path import expanduser
from langchain.llms import LlamaCpp
from langchain_experimental.chat_models import Llama2Chat

In [9]:
from langchain.prompts import PromptTemplate

#### Load Model

In [10]:
# !wget "https://huggingface.co/TheBloke/LLaMa-7B-GGML/resolve/main/llama-7b.ggmlv3.q2_K.bin"
# this is because llama.cpp no longer supports GGML
# models as of August 21st. GGML has been replaced by
# a new format called GGUF.
!wget "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q3_K_L.gguf"

--2023-11-20 17:39:27--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q3_K_L.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.55, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/ddb3c7858d89128bdcbb441d6636510061b4babe2498d3ab865253967e141707?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q3_K_L.gguf%3B+filename%3D%22llama-2-7b-chat.Q3_K_L.gguf%22%3B&Expires=1700761167&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDc2MTE2N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlL2RkYjNjNzg1OGQ4OTEyOGJkY2JiNDQxZDY2MzY1MTAwNjFi

In [65]:
# !wget "https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q5_K_M.gguf"

--2023-11-20 17:29:40--  https://huggingface.co/TheBloke/zephyr-7B-beta-GGUF/resolve/main/zephyr-7b-beta.Q5_K_M.gguf
Resolving huggingface.co (huggingface.co)... 65.8.178.12, 65.8.178.118, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/fe/17/fe17596731f84a0d03bece77489780bc7e068323c0aeca88b6393d3e9e65dd49/37894e5b171bd7228f4af7bd5bb0758dd29d6f07fb8e4742e387720f66bac434?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27zephyr-7b-beta.Q5_K_M.gguf%3B+filename%3D%22zephyr-7b-beta.Q5_K_M.gguf%22%3B&Expires=1700760580&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDc2MDU4MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ZlLzE3L2ZlMTc1OTY3MzFmODRhMGQwM2JlY2U3NzQ4OTc4MGJjN2UwNjgzMjNjMGFlY2E4OGI2MzkzZDNlOWU2NWRkNDkvMzc4OTRlNWIxNzFiZDcyMjhmNGFmN2JkNWJiMDc1OGRkMj

In [11]:
mkdir models

In [ ]:
# mv llama-7b.ggmlv3.q2_K.bin ./models/

In [12]:
!mv llama-2-7b-chat.Q3_K_L.gguf ./models/

In [66]:
!mv zephyr-7b-beta.Q5_K_M.gguf ./models/

In [13]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d833d1c9db1dc4f1cec221b6cc67f75d8e4881fde9620b892fe8328a8b540e0a
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [14]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

#### Load Data

In [15]:
!unzip ethinic_articles.zip

Archive:  ethinic_articles.zip
  inflating: doc_3.txt               
  inflating: doc_1.txt               
  inflating: doc_2.txt               


In [40]:
!unzip ethinic_catalogue.zip

Archive:  ethinic_catalogue.zip
   creating: content/productcatalog/
  inflating: content/productcatalog/ Janasya Women s White Embroidered Cotton Kurta .csv  
  inflating: content/productcatalog/ Janasya Women s Black Cotton Kurta with Pocket .csv  
  inflating: content/productcatalog/ Zeel Clothing Women s Organza Semi stitched Lehenga Choli 7609 Floral Black Wedding Lehenga_Black_Free Size .csv  
  inflating: content/productcatalog/ INDO ERA Women s Embroidered Viscose Straight Kurta Trouser with Dupatta Set RRRRR3718 .csv  
  inflating: content/productcatalog/ Zeel Clothing Women s Organza Floral White Semi Stitched Lehenga Choli 7611 Wedding Floral Lehenga Latest White .csv  
  inflating: content/productcatalog/ PURVAJA Women s Faux Silk Semi stitched Lehenga Choli Red Bandhani_Red_Free Size .csv  
  inflating: content/productcatalog/ See Designs Men s Cotton Blend Kurta Pyjama Set .csv  
  inflating: content/productcatalog/ rangita Pink Rayon Printed Calf Length Front Slit Festiv

## Process the Fashion Product Catalogue


In [43]:
!unzip catalogue.zip

Archive:  catalogue.zip
  inflating: myntra_products_catalog.csv  


In [44]:
catalogue = pd.read_csv('./myntra_products_catalog.csv')

In [45]:
products = []
for product_data in catalogue.values:
  products.append(" ".join(str(x) for x in product_data[1:]))

In [49]:
products[0]

'DKNY Unisex Black & Grey Printed Medium Trolley Bag DKNY Unisex 11745 7 Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer  Black'

In [50]:
# processed_products = []
# for line in products:
#   processed_products.append(" ".join([w for w in line.lower()]))

In [53]:
product_embeddings = embed_model.embed_documents(products)

## Process the Fashion Blog Articles

In [16]:
loader = DirectoryLoader('./', glob="*.txt", show_progress=True, loader_cls = TextLoader)
docs = loader.load()

100%|██████████| 3/3 [00:00<00:00, 3421.13it/s]


In [17]:
len(docs)

3

In [18]:
docs

[Document(page_content=',0\n0,"Indian Traditional Dress — What do Indian Women and Men WearDive into the vibrant world of Indian traditional attire! This intriguing realm encompasses an array of clothing worn by women and men, reflecting India’s rich cultural heritage. From a woman’s elegant saree to a man’s distinguished sherwani, you’ll discover enchanting beauty in these diverse outfits at our online store, your window into authentic Indian fashion.Let’s talk about the fascinating variety of outfits worn by Indian women. The sheer diversity in these traditional ensembles truly mirrors India’s multicultural tapestry. Take, for instance, the timeless saree.A long piece of fabric is available in countless hues and styles reserved for diverse occasions. While it has roots tracing back to ancient civilizations, its appeal remains undiminished today. Then there’s the Lehenga Choli, originating from Gujarat and Rajasthan regions. It is an attire synonymous with celebration due to its rich 

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

In [20]:
all_splits

[Document(page_content=',0', metadata={'source': 'doc_3.txt'}),
 Document(page_content='0,"Indian Traditional Dress — What do Indian Women and Men WearDive into the vibrant world of Indian traditional attire! This intriguing realm encompasses an array of clothing worn by women and men, reflecting India’s rich cultural heritage. From a', metadata={'source': 'doc_3.txt'}),
 Document(page_content='heritage. From a woman’s elegant saree to a man’s distinguished sherwani, you’ll discover enchanting beauty in these diverse outfits at our online store, your window into authentic Indian fashion.Let’s talk about the fascinating variety of outfits', metadata={'source': 'doc_3.txt'}),
 Document(page_content='variety of outfits worn by Indian women. The sheer diversity in these traditional ensembles truly mirrors India’s multicultural tapestry. Take, for instance, the timeless saree.A long piece of fabric is available in countless hues and styles', metadata={'source': 'doc_3.txt'}),
 Document(page

## Store the Article Embeddings in Vector DB

In [21]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)

## Perform Similarity Search on Input

In [22]:
question = " I like traditional clothing and have frugal spending habits. Suggest an outfit for Diwali celebration"
relevant_docs = vectorstore.similarity_search(question)

In [23]:
relevant_docs

[Document(page_content='5,"The festive season brings a lot of excuses to buy your favorite outfits. And if it’s Diwali we are talking about, it makes sense to explore stylish men’s ethnic outfits. An Indian man looks dapper in a kurta pajama or a sherwani. The custom of', metadata={'source': 'doc_1.txt'}),
 Document(page_content='find festive deals on your favorite outfits.https://blog.indianweddingsaree.com/mens-ethnic-outfits-that-are-trending-this-festive-season"', metadata={'source': 'doc_1.txt'}),
 Document(page_content='you were wearing a dhoti while visiting temples or religious places until now, you can step ahead and don a stylish ethnic look this Diwali.You can choose a matching dhoti kurta or create contrast. Both are fine. With a maroon kurta, you can choose', metadata={'source': 'doc_1.txt'}),
 Document(page_content='get started knowing what collection is trending this festive season and what you can wear during this festive season.Floral Kurta PajamaMen prefer a kurta paj

In [24]:
relevant_docs[0]

Document(page_content='5,"The festive season brings a lot of excuses to buy your favorite outfits. And if it’s Diwali we are talking about, it makes sense to explore stylish men’s ethnic outfits. An Indian man looks dapper in a kurta pajama or a sherwani. The custom of', metadata={'source': 'doc_1.txt'})

## Setup LLM Model

In [ ]:
# LlamaCpp?

In [25]:
model_path = expanduser("/content/models/llama-2-7b-chat.Q3_K_L.gguf")
# model_path = expanduser("/content/models/zephyr-7b-beta.Q5_K_M.gguf")

llm = LlamaCpp(
    model_path=model_path,
    streaming=False,
    # n_ctx=2048
    temperature=0.75,
    max_tokens=1000,
    top_p=1,
    # callback_manager=callback_manager,
)
model = Llama2Chat(llm=llm)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [26]:
user_prompt = "I like traditional clothing and have frugal spending habits. Suggest an outfit for Diwali celebration"
# user_content = "chickens"

In [27]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [28]:
prompt_template = PromptTemplate.from_template(template)

In [29]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

In [38]:
import textwrap

In [30]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)
# chain = LLMChain(llm=model, prompt=prompt_template)

In [31]:
# create retriever from vectorDB
retriever = vectorstore.as_retriever()

In [32]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

In [33]:
from langchain.schema.output_parser import StrOutputParser

In [34]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | model
    | StrOutputParser()
)

## Run Model for Inference

In [88]:
# chain.invoke("I like traditional clothing and have frugal spending habits. Suggest an outfit for Diwali celebration")

KeyboardInterrupt: ignored

In [61]:
user_query = "I like traditional clothing and have frugal spending habits. Suggest an outfit for Diwali celebration"

In [35]:
LLM_output = chain.invoke("I like traditional clothing and have frugal spending habits. Suggest an outfit for Diwali celebration")

In [36]:
LLM_output

"  Of course, I'd be happy to help! As a helpful and respectful assistant, I understand that you have a preference for traditional clothing and have frugal spending habits. Based on the provided context, here are some suggestions for an outfit that is both stylish and budget-friendly for Diwali celebrations:\n1. Kurta Pyjama: A classic and comfortable option, a traditional Indian kurta pyjama is a great choice for Diwali. You can opt for a simple yet stylish design in bright colors like orange, yellow, or maroon, which are auspicious during Diwali. Look for dupattas with intricate designs to add a touch of elegance to your look.\n2. Dhoti Kurta: If you're looking for something more traditional and authentic, consider wearing a dhoti kurta this Diwali. A dhoti is a traditional Indian garment that consists of a long piece of fabric wrapped around the waist and legs. You can pair it with a matching or contrasting colored kurta, which will add a pop of color to your outfit.\n3. Sherwani: F

In [39]:
textwrap.wrap(LLM_output)

["  Of course, I'd be happy to help! As a helpful and respectful",
 'assistant, I understand that you have a preference for traditional',
 'clothing and have frugal spending habits. Based on the provided',
 'context, here are some suggestions for an outfit that is both stylish',
 'and budget-friendly for Diwali celebrations: 1. Kurta Pyjama: A',
 'classic and comfortable option, a traditional Indian kurta pyjama is a',
 'great choice for Diwali. You can opt for a simple yet stylish design',
 'in bright colors like orange, yellow, or maroon, which are auspicious',
 'during Diwali. Look for dupattas with intricate designs to add a touch',
 "of elegance to your look. 2. Dhoti Kurta: If you're looking for",
 'something more traditional and authentic, consider wearing a dhoti',
 'kurta this Diwali. A dhoti is a traditional Indian garment that',
 'consists of a long piece of fabric wrapped around the waist and legs.',
 'You can pair it with a matching or contrasting colored kurta, which',
 '

In [56]:
from sentence_transformers import util

In [58]:
def getProduct(data):
  # , check):
  # category = str(data[3])
  # if ((category == "Men" or category == "Boys") and check is False) or ((category == "Women"  or category == "Girls") and check is True):
  #   return

  print("  Product: " + str(data[1]))
  print("  " + str(data[2]) + " | " + str(data[3]))
  print("  Color:" + str(data[7]))
  print("  Description: ")
  for line in textwrap.wrap(str(data[6])):
    print(line)
  print("  --------------------------------")

In [64]:
def processUserRequest(input, query, limit = 0.3):

  # ire_input = " ".join([w for w in word_tokenize((query + output).lower()) if w not in stop])
  query_embedding = embed_model.embed_query(input)
  # query_embedding = model.encode(ire_input)


  sim_vals = []
  for embedding in product_embeddings:
    sim_vals.append(util.cos_sim(query_embedding, embedding)[0].numpy()[0])

  indices = []
  for i in np.argsort(sim_vals)[-5:][-1::-1]:
    if(sim_vals[i] > limit):
      indices.append(i)

  text_output = ""

  # print("\nCustomer Profile: \n")
  # for line in textwrap.wrap(profile):
  #   print(line)
  print("\nQuery: ")
  for line in textwrap.wrap(query):
    print(line)

  print("\nLLM Output: ")
  for line in textwrap.wrap(input):
    print(line)

  count = 0

  print("\nHere are relevant products for you to check out:")
  # get the data for each of the result
  for i in indices:
    getProduct(catalogue.values[i])
    # getProduct(catalogue.values[i], check)
    count += 1

  if count == 0:
    print("\nNo relevant results found")



In [65]:
processUserRequest(LLM_output, user_query, limit = 0.3)


Query: 
I like traditional clothing and have frugal spending habits. Suggest
an outfit for Diwali celebration

LLM Output: 
  Of course, I'd be happy to help! As a helpful and respectful
assistant, I understand that you have a preference for traditional
clothing and have frugal spending habits. Based on the provided
context, here are some suggestions for an outfit that is both stylish
and budget-friendly for Diwali celebrations: 1. Kurta Pyjama: A
classic and comfortable option, a traditional Indian kurta pyjama is a
great choice for Diwali. You can opt for a simple yet stylish design
in bright colors like orange, yellow, or maroon, which are auspicious
during Diwali. Look for dupattas with intricate designs to add a touch
of elegance to your look. 2. Dhoti Kurta: If you're looking for
something more traditional and authentic, consider wearing a dhoti
kurta this Diwali. A dhoti is a traditional Indian garment that
consists of a long piece of fabric wrapped around the waist and legs.
Y

In [40]:
# prompt_template = PromptTemplate.from_template(
#     "[INST] <<SYS>> You are a helpful, respectful and honest assistant. \
#     Always answer as helpfully as possible, while being safe. If you don't know \
#     the answer to a question, please don't share false information.\
#     <</SYS>>{prompt}[/INST]"
# )
# prompt_template.format(prompt=template)

In [ ]:
# from langchain.prompts.chat import (
#     ChatPromptTemplate,
#     HumanMessagePromptTemplate,
#     MessagesPlaceholder,
# )
# from langchain.schema import SystemMessage

# template_messages = [
#     SystemMessage(content="You are a helpful assistant."),
#     MessagesPlaceholder(variable_name="chat_history"),
#     HumanMessagePromptTemplate.from_template("{text}"),
# ]
# prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [66]:
print(
    chain.run(
        text="I like traditional clothing and have frugal spending habits. Suggest an outfit for Diwali celebration"
    )
)

AttributeError: ignored

In [ ]:
# n_gpu_layers = 32  # Metal set to 1 is enough.
# n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# #llama = LlamaCppEmbeddings(model_path="/data/llama.cpp/models/llama-2-7b-chat/ggml-model-q4_0.bin")
# llm = LlamaCpp(
#     model_path="/data/llama.cpp/models/llama-2-7b-chat/ggml-model-q4_0.bin",
#     n_gpu_layers=n_gpu_layers,
#     n_batch=n_batch,
#     n_ctx=2048,
#     f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
#     callback_manager=callback_manager,
#     verbose=False,
# )

ValidationError: ignored

In [ ]:
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [ ]:
LLMChain?

In [ ]:
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path="./models/llama-7b.ggmlv3.q2_K.bin")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [ ]:
llm.create_completion(
    prompt = "Q: " + profile1 + " Suggest something to wear for birthday party. A: ",
    max_tokens = 256,
    stop = ["Q:"],
)

NameError: ignored